In [1]:
import uuid
import copy

In [2]:
# max num of pointers in a leaf
size = 4

In [3]:
class Leaf:
    def __init__(self, depth=0):
        self.uid = uuid.uuid4()
        self.depth = depth
        self.node = [-1 for i in range(size+size-1)] # pointer, key, pointer, ...
        self.parentId = copy.deepcopy(self.uid)
        
    def getKeys(self):
        return [x for i, x in enumerate(self.node) if i % 2 == 1]        
    
    def getPointers(self):
        return [x for i, x in enumerate(self.node) if i % 2 == 0]
        
    def getNextLeafId(self, key, parent_max_key):
        bigger_keys = list(filter(lambda x: key < x, self.getKeys()))
        if len(bigger_keys) >= 1:
            return self.node[self.node.index(bigger_keys[0])-1]
        else:
            same_key = list(filter(lambda x: key == x, self.getKeys()))
            if len(same_key) == 1:
                return self.node[self.node.index(key)+1]
            if key <= parent_max_key:
                return self.node[self.node.index(max(self.getKeys()))+1]
            # valid_pointers = list(filter(lambda x: x != -1, self.pointers))
            valid_pointers = list(filter(lambda x: x != -1, self.getPointers()))
#             print('vp:', valid_pointers)
            if len(valid_pointers) == 0: return -1
            return valid_pointers[-1]
        
    def showPointer(self, key):
        # return self.node[2*self.getKeys().index(key)+1]
        return self.getKeys().index(key)

In [4]:
class Tree:
    def __init__(self):
        self.depth = 1
        root = Leaf(self.depth)
        self.rootId = root.uid
        self.leaves = [root] # make different object
        
    def addLeaf(self, depth=0):
        new_leaf = Leaf(depth)
#         print('add leaf:', new_leaf.uid)
        self.leaves.append(new_leaf)
        return new_leaf
        
    def getLeaf(self, uid):
        return list(filter(lambda l: l.uid == uid, self.leaves))[0]
    
    def leafFor(self, key):
        pointer = self.rootId
        cnt = 0
        parent_max_key = float('inf')
        if self.depth == 1: return self.getLeaf(self.rootId)
        while cnt <= 2*self.depth-1:
            cnt += 1
            leaf = self.getLeaf(pointer)
            next_pointer = self.getLeaf(pointer).getNextLeafId(key, parent_max_key)
            parent_max_key = max(leaf.getKeys())
            if type(next_pointer) != uuid.UUID: break
            pointer = next_pointer
#         for i in range(self.depth):
#             if i < self.depth-1:
#                 pointer = self.getLeaf(pointer).getNextLeafId(key)
#                 print('pointer now:', pointer)
#         print('leaf to:', pointer)
        return self.getLeaf(pointer)
    
    def insertInLeaf(self, leaf, key, pointer, temp=False):
#         print('node now:', leaf.node)
        if leaf.getKeys()[0] == -1 and leaf.getPointers()[0] == -1:
#             print('leaf initial -1')
            leaf.node[1] = key
            leaf.node[0] = pointer
            return
        elif key < leaf.getKeys()[0]:
#             print('insert to initial:', key, '<', leaf.getKeys()[0])
            first_pointer = leaf.node[0]
            if ( type(first_pointer) == uuid.UUID and 
                    max(self.getLeaf(first_pointer).getKeys()) < key ):
#                 print('skip first')
                leaf.node.insert(1, key)
                leaf.node.insert(2, pointer)
            else:
#                 print('simple first insert')
                leaf.node.insert(0, pointer)
                leaf.node.insert(1, key)
            if not temp:
                nulls = [i for i, x in enumerate(leaf.node) if x == -1]
                if len(nulls) >= 2:
                    null_index = leaf.node.index(-1)
                    del leaf.node[null_index:null_index+2]
                else: 
                    leaf.node = leaf.node[:-2]
        else:
#             print('key:', key)
#             print(list(filter(lambda k: k <= key and k != -1, leaf.getKeys())))
            key_before = list(filter(lambda k: k <= key and k != -1, leaf.getKeys()))[-1]
            idx = 2*leaf.getKeys().index(key_before)+1
#             print('kbi:', key, key_before, idx)
            if idx+2 < 2*size-1 and leaf.node[idx+1] == -1 and leaf.node[idx+2] == -1:
#                 print('simple replace')
                leaf.node[idx+1] = pointer
                leaf.node[idx+1+1] = key
                return
#             elif ( idx-1 != -1 and idx+1 != -1 and type(leaf.node[idx+1]) == uuid.UUID 
#                   and type(leaf.node[idx+2]) == uuid.UUID ):
            elif ( idx+1 < len(leaf.node) and type(leaf.node[idx-1]) == uuid.UUID 
                  and type(leaf.node[idx+1]) == uuid.UUID ):
#                 print('add last')
                leaf.node.insert(idx+2, key)
                leaf.node.insert(idx+3, pointer)
            else:
#                 print('insert last')
                leaf.node.insert(idx+1, pointer)
                leaf.node.insert(idx+1+1, key)
            if not temp:
                nulls = [i for i, x in enumerate(leaf.node) if x == -1]
                if len(nulls) >= 2:
                    null_index = leaf.node.index(-1)
                    del leaf.node[null_index:null_index+2]
                else: 
                    leaf.node = leaf.node[:-2]
#         print('inst res:', leaf.getKeys())
#         print('inst res:', leaf.getPointers())
            
    def insertInParent(self, leaf, key, pointer):
        if leaf.uid == self.rootId:
            self.depth += 1
#             print('this is root')
            new_root = self.addLeaf(self.depth)
#             print('new root uid:', new_root.uid)
            new_root.node[0] = leaf.uid
            new_root.node[1] = key
            new_root.node[2] = pointer
            old_root = self.getLeaf(self.rootId)
            old_root.parentId = new_root.uid
            new_leaf = self.getLeaf(pointer)
#             print('new leaf uid2:', new_leaf.uid)
            new_leaf.parentId = new_root.uid
            old_root.node[-1] = new_leaf.uid
            self.rootId = new_root.uid
        else:
#             print('parent id:', leaf.parentId)
            parent_leaf = self.getLeaf(leaf.parentId)
            if len(list(filter(lambda x: x != -1, parent_leaf.getPointers()))) < size:
#                 print('parent safe')
                child_leaf = self.getLeaf(pointer)
                child_leaf.parentId = parent_leaf.uid
#                 print('parent keys:', parent_leaf.getKeys())
#                 print('child keys:', child_leaf.getKeys())
#                 print('parent pointers:', parent_leaf.getPointers())
#                 print('child pointers:', child_leaf.getPointers())
                self.insert(key, pointer, parent_leaf)
            else:
#                 print('parent full')
                temp_parent = copy.deepcopy(parent_leaf) # make different object
                # insert (K', N') ...
                self.insertInLeaf(temp_parent, key, pointer, True)
                # Erase all ...
                parent_leaf.node = [-1 for i in range(size+size-1)]
                #parent_leaf.node[-1] = temp_parent.node[-1]
                # Create ...
                new_parent = self.addLeaf(parent_leaf.depth)
                new_parent.parentId = parent_leaf.parentId
                leaf.parentId = new_parent.uid
                self.getLeaf(pointer).parentId = new_parent.uid
                # Copy T.P1, ...
                # part = 2*((size-1+1)//2+1)
                part = (2*size)//2+1
                parent_leaf.node[:part] = temp_parent.node[:part]
                # Let K'' ...
                mid_key = temp_parent.node[part]
#                 print('mid_key:', mid_key)
                # Copy T.P[(n+1)/2]+1, ...
                new_parent.node[:len(temp_parent.node[part+1:])] \
                    = temp_parent.node[part+1:]
#                 print('am I root?', parent_leaf.uid == self.rootId)
                self.insertInParent(parent_leaf, mid_key, new_parent.uid)
                
    def insert(self, key, pointer, _leaf=0):
        if _leaf == 0:
            leaf = self.leafFor(key)
#             print('selected leaf:', leaf.uid)
#             print('with keys:', leaf.getKeys())
#             print('and pointers:', leaf.getPointers())
        else: 
            leaf = _leaf
        if len(list(filter(lambda x: x != -1, leaf.getKeys()))) < size-1:
#             print('this leaf has space')
            self.insertInLeaf(leaf, key, pointer)
        else:
            new_leaf = self.addLeaf(leaf.depth)
            new_leaf.parentId = leaf.parentId
#             print('to copy:', leaf.getKeys())
            temp_leaf = copy.deepcopy(leaf) # make different object
            temp_leaf.node = temp_leaf.node[:-1]
            self.insertInLeaf(temp_leaf, key, pointer, True)
#             print('temp keys:', temp_leaf.getKeys())
#             print('temp pos:', temp_leaf.getPointers())
            # Set L'.Pn = L.Pn
            new_leaf.node[-1] = leaf.node[-1] 
            # Set L.Pn = L'
#             print('new leaf uid:', new_leaf.uid)
#             print('new leaf node:', new_leaf.node)
            leaf.node[-1] = new_leaf.uid
            # Erase ...
            leaf.node[:-1] = [-1 for i in range(size+size-1-1)]
#             print('after erased:', new_leaf.node)
            # Copy T.P1, ...
            # part = 2*((size-1)//2+1)+1
            part = (2*size-1)//2+1
            leaf.node[:part] = temp_leaf.node[:part]
            # Copy T.P[n/2]+1, ...
            new_leaf.node[:len(temp_leaf.node[part:])] = temp_leaf.node[part:]
            min_key = new_leaf.node[1]
#             print('min_key:', min_key)
#             print('leaf keys:', leaf.getKeys())
#             print('leaf pointers:', leaf.getPointers())
#             print('new leaf keys:', new_leaf.getKeys())
#             print('new leaf pointers:', new_leaf.getPointers())
#             print('with root id:', self.rootId)
            self.insertInParent(leaf, min_key, new_leaf.uid)

In [5]:
tree = Tree()
# data = [0]
# data = [i for i in range(4)]
# data = [i for i in range(6)]
# data = [i for i in range(15)]
# data = [3, 1, 0, 2]
# data = [3, 1, 4, 0, 2]
data = [5, 3, 1, 4, 0, 2]

In [6]:
for i in range(len(data)):
#     print('insert:', data[i])
    tree.insert(data[i], i)
#     print()

In [7]:
for i in range(tree.depth, 0, -1):
    leaves = list(filter(lambda x: x.depth == i, tree.leaves))
    if len(leaves) == 1:
        print(leaves[0].getKeys())
    else:
        sorted_leaves = sorted(leaves, key=lambda l: l.getKeys()[0])
        to_print = []
        for i, leaf in enumerate(sorted_leaves):
            to_print.append(str(leaf.getKeys()))
            if i == len(sorted_leaves)-1: break
            # print(leaf.getPointers()[-1], sorted_leaves[i+1].uid)
            if leaf.getPointers()[-1] == sorted_leaves[i+1].uid:
                to_print.append('->')
        print(''.join(to_print))

[2, 4, -1]
[0, 1, -1]->[2, 3, -1]->[4, 5, -1]


In [8]:
to_print = []
for i in data:
    leaf = tree.leafFor(i)
    to_print.append(leaf.getPointers()[leaf.showPointer(i)])

print(to_print)
print(data)

[0, 1, 2, 3, 4, 5]
[5, 3, 1, 4, 0, 2]


In [9]:
# leaf = tree.leafFor(5)
# leaf = tree.leaves[3]
leaf = tree.getLeaf(tree.rootId)
print('keys:', leaf.getKeys())
print('poi:', leaf.getPointers())
print('uid:', leaf.uid)
print('pid:', leaf.parentId)
print('rootId:', tree.rootId)

keys: [2, 4, -1]
poi: [UUID('a6aece2f-b5da-4444-8b27-e4cbaaa20aca'), UUID('335d63cf-0e9b-43cc-881a-3f8a799c167c'), UUID('49218ce3-db71-4c51-a3cf-e2999d523658'), -1]
uid: 24921413-2bf0-447b-a76f-e04f1a13ad0a
pid: 24921413-2bf0-447b-a76f-e04f1a13ad0a
rootId: 24921413-2bf0-447b-a76f-e04f1a13ad0a


In [10]:
tree2 = Tree()
# data = [0]
# data = [i for i in range(4)]
# data = [i for i in range(6)]
data = [i for i in range(15)]
# data = [3, 1, 0, 2]
# data = [3, 1, 4, 0, 2]
# data = [5, 3, 1, 4, 0, 2]

In [11]:
for i in range(len(data)):
#     print('insert:', data[i])
    tree2.insert(data[i], i)
#     print()

In [12]:
for i in range(tree2.depth, 0, -1):
    leaves = list(filter(lambda x: x.depth == i, tree2.leaves))
    if len(leaves) == 1:
        print(leaves[0].getKeys())
    else:
        sorted_leaves = sorted(leaves, key=lambda l: l.getKeys()[0])
        to_print = []
        for i, leaf in enumerate(sorted_leaves):
            to_print.append(str(leaf.getKeys()))
            if i == len(sorted_leaves)-1: break
            # print(leaf.getPointers()[-1], sorted_leaves[i+1].uid)
            if leaf.getPointers()[-1] == sorted_leaves[i+1].uid:
                to_print.append('->')
        print(''.join(to_print))

[6, -1, -1]
[2, 4, -1]->[8, 10, 12]
[0, 1, -1]->[2, 3, -1]->[4, 5, -1]->[6, 7, -1]->[8, 9, -1]->[10, 11, -1]->[12, 13, 14]


In [13]:
to_print = []
for i in data:
    leaf = tree2.leafFor(i)
    to_print.append(leaf.getPointers()[leaf.showPointer(i)])

print(to_print)
print(data)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [14]:
# leaf = tree2.leafFor(5)
# leaf = tree2.leaves[3]
leaf2 = tree2.getLeaf(tree2.rootId)
print('keys:', leaf2.getKeys())
print('poi:', leaf2.getPointers())
print('uid:', leaf2.uid)
print('pid:', leaf2.parentId)
print('rootId:', tree2.rootId)

keys: [6, -1, -1]
poi: [UUID('bbb8a4a4-7214-4321-87c5-a93dde7e6455'), UUID('62b8ac1b-8069-41d6-852f-751c51a602c5'), -1, -1]
uid: abc21270-ff86-446a-94f7-e32ad2ce1c4c
pid: abc21270-ff86-446a-94f7-e32ad2ce1c4c
rootId: abc21270-ff86-446a-94f7-e32ad2ce1c4c
